In [129]:
# pandas and numpy
import pandas as pd
import numpy as numpy

# punctuation, stop words and English language model
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
nlp = en_core_web_sm.load()

# textblob
from textblob import TextBlob

# countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# gensim
import gensim
from gensim import models

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [130]:
congress_tweets = pd.read_csv("116th Congressional Tweets and Demographics.csv")
congress_tweets = congress_tweets.dropna(subset = ['text']).reset_index()
# fill in this line of code with a sufficient number of tweets, depending on your computational resources
congress_tweets.head()

,index,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share
0,0,1.081010e+18,RepByrne,2019-01-03T21:23:00-05:00,Great news for Baldwin County! The economy of ...,Bradley Byrne,Rep,8-Jan-14,2/16/1955,M,AL,1,Republican,"1,318,255","729,547","795,696","1,255,925"
1,1,1.080880e+18,RepByrne,2019-01-03T12:30:38-05:00,Outstanding news today from @Airbus! @JetBlue ...,Bradley Byrne,Rep,8-Jan-14,2/16/1955,M,AL,1,Republican,"1,318,255","729,547","795,696","1,255,925"
2,2,1.080830e+18,RepByrne,2019-01-03T09:12:07-05:00,RT @senatemajldr Democrats will have to get se...,Bradley Byrne,Rep,8-Jan-14,2/16/1955,M,AL,1,Republican,"1,318,255","729,547","795,696","1,255,925"
3,3,1.080890e+18,RepByrne,2019-01-03T13:20:53-05:00,Here is a sign of things to come: As Democrats...,Bradley Byrne,Rep,8-Jan-14,2/16/1955,M,AL,1,Republican,"1,318,255","729,547","795,696","1,255,925"
4,4,1.080870e+18,RepByrne,2019-01-03T12:10:26-05:00,Let's understand what we're dealing with here:...,Bradley Byrne,Rep,8-Jan-14,2/16/1955,M,AL,1,Republican,"1,318,255","729,547","795,696","1,255,925"


In [131]:
congress_tweets = congress_tweets.sample(frac=0.001, replace = True, random_state=1)
congress_tweets.head()

,index,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share
128037,128037,1.268690e+18,RepTjCox,2020-06-04T20:00:00-04:00,"As the #1 Dairy District in the country, we ar...",TJ Cox,Rep,3-Jan-19,7/18/1963,M,CA,21,Democrat,"4,483,814","8,753,792","7,854,285","4,839,958"
491755,491755,1.275140e+18,RepSusieLee,2020-06-22T14:53:14-04:00,RT @WesleyWhistle DeVos made it harder for stu...,Susie Lee,Rep,3-Jan-19,11/7/1966,F,NV,3,Democrat,"512,058","539,260","531,373","463,567"
470924,470924,1.239640e+18,USRepLong,2020-03-16T15:52:18-04:00,RT @GovParsonMO Get the latest CDC guidance on...,Billy Long,Rep,3-Jan-11,8/11/1955,M,MO,7,Republican,"1,594,511","1,071,068","1,223,796","1,482,440"
791624,791624,1.199040e+18,JohnCornyn,2019-11-25T14:13:18-05:00,RT @libbyhamb Sen. @JohnCornyn volunteers at @...,John Cornyn,Sen,2-Dec-02,2/2/1952,M,TX,Senate,Republican,"4,685,047","3,877,868","3,308,124","4,569,843"
491263,491263,1.263200e+18,RepSusieLee,2020-05-20T15:58:37-04:00,Our country hasn’t faced an unemployment crisi...,Susie Lee,Rep,3-Jan-19,11/7/1966,F,NV,3,Democrat,"512,058","539,260","531,373","463,567"


In [121]:
nlp = en_core_web_sm.load()
##cfpb['tokens'] = cfpb['Consumer complaint narrative'].map(lambda x: rem_punc_stop(x))
#congress_tweets['text_parsed'] = congress_tweets['text'].apply(nlp)

In [122]:
def rem_punc_stop(text):
    stop_words = STOP_WORDS
    punc = set(punctuation)
    
    punc_free = "".join([ch for ch in text if ch not in punc])
    
    doc = nlp(punc_free)
    
    spacy_words = [token.text for token in doc]
    
    spacy_words = [word for word in spacy_words if not word.startswith('http')]
    
    no_punc = [word for word in spacy_words if word not in stop_words]
    
    return no_punc

In [123]:
congress_tweets['text'] = congress_tweets['text'].map(lambda x: rem_punc_stop(x))

#tokens_reduced = rem_punc_stop(text)
#congress_tweets['text_parsed_new'] = rem_punc_stop(congress_tweets['text_parsed'])
#congress_tweets['text'] =congress_tweets['text'].apply(rem_punc_stop)

In [137]:
## remove emojis
import spacy, re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags=re.UNICODE)

NameError: name 're' is not defined

In [124]:
congress_tweets.head()

,index,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share
128037,128037,1.268690e+18,RepTjCox,2020-06-04T20:00:00-04:00,"[As, 1, Dairy, District, country, proud, celeb...",TJ Cox,Rep,3-Jan-19,7/18/1963,M,CA,21,Democrat,"4,483,814","8,753,792","7,854,285","4,839,958"
491755,491755,1.275140e+18,RepSusieLee,2020-06-22T14:53:14-04:00,"[RT, WesleyWhistle, DeVos, harder, students, l...",Susie Lee,Rep,3-Jan-19,11/7/1966,F,NV,3,Democrat,"512,058","539,260","531,373","463,567"
470924,470924,1.239640e+18,USRepLong,2020-03-16T15:52:18-04:00,"[RT, GovParsonMO, Get, latest, CDC, guidance, ...",Billy Long,Rep,3-Jan-11,8/11/1955,M,MO,7,Republican,"1,594,511","1,071,068","1,223,796","1,482,440"
791624,791624,1.199040e+18,JohnCornyn,2019-11-25T14:13:18-05:00,"[RT, libbyhamb, Sen, JohnCornyn, volunteers, C...",John Cornyn,Sen,2-Dec-02,2/2/1952,M,TX,Senate,Republican,"4,685,047","3,877,868","3,308,124","4,569,843"
491263,491263,1.263200e+18,RepSusieLee,2020-05-20T15:58:37-04:00,"[Our, country, faced, unemployment, crisis, li...",Susie Lee,Rep,3-Jan-19,11/7/1966,F,NV,3,Democrat,"512,058","539,260","531,373","463,567"


In [132]:
bow_vector = CountVectorizer(tokenizer = rem_punc_stop, ngram_range=(1,1))

In [133]:
bow_matrix = bow_vector.fit_transform(congress_tweets['text'])

In [134]:
bow_matrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [135]:
feature_names = bow_vector.get_feature_names()
feature_names[0:10]

['\n', '\n\n', '\n\n ', '\n ', '\n \n', '\n\xa0\n', ' ', ' \n', ' \n\n', '  ']